In [ ]:
from dataclasses import dataclass, field, replace
from re import L
from typing import List, Dict, Optional, Tuple
from Pynite import FEModel3D
from Pynite.Rendering import Renderer
import numpy as np
import pandas as pd
from itertools import product
from tqdm.auto import tqdm


class Naming:
    @staticmethod
    def node(name: str, end: str, floor: bool = False) -> str:
        prefix = 'floor ' if floor else ''
        return f'{prefix}{name}{end}'

    @staticmethod
    def member(name: str) -> str:
        return name

    @staticmethod
    def wall(p1_name: str, p2_name: str, side: str) -> str:
        return f'{side} wall {p1_name}-{p2_name}'

    @staticmethod
    def side_wall(side: str) -> str:
        return f'{side} wall'
    
    @staticmethod
    def corner_node(corner: str, floor: bool = False) -> str:
        level = 'floor' if floor else 'ceiling'
        return f'corner_{corner}_{level}'

    @staticmethod
    def header_node(end: str) -> str:
        return f'header_{end}'

    @staticmethod
    def header_member() -> str:
        return 'header'

    @staticmethod
    def tail_header_node(tail_name: str) -> str:
        return f'{tail_name}_header'


@dataclass
class DesignParameters:
    room_length: float
    room_width: float
    room_height: float
    plank_thickness: float
    opening_width: float
    opening_length: float
    opening_x_start: float
    wall_beam_contact_depth: float
    live_load_mpa: float
    wall_thickness: float

    @property
    def floor_to_floor(self):
        return self.room_height / 2 + self.plank_thickness / 2

    @property
    def beam_length(self):
        return self.room_width + self.wall_beam_contact_depth

def load_design_parameters(path: str) -> DesignParameters:
    try:
        params_df = pd.read_csv(path)
        params = params_df.iloc[0].to_dict()
        return DesignParameters(**params)
    except FileNotFoundError:
        raise SystemExit(f"Error: Design parameters file not found at '{path}'")
    except Exception as e:
        raise SystemExit(f"Error loading design parameters from '{path}': {e}")

def load_material_strengths(path: str) -> Dict:
    try:
        materials_df = pd.read_csv(path)
        materials_df.set_index('material', inplace=True)
        return materials_df.to_dict(orient='index')
    except FileNotFoundError:
        raise SystemExit(f"Error: Material strengths file not found at '{path}'")
    except Exception as e:
        raise SystemExit(f"Error loading material strengths from '{path}': {e}")

def load_beam_catalog(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path)
    except FileNotFoundError:
        raise SystemExit(f"Error: Beam catalog file not found at '{path}'")
    except Exception as e:
        raise SystemExit(f"Error loading beam catalog from '{path}': {e}")

# Units are mm, N, and MPa (N/mm²)
DEFAULT_PARAMS = load_design_parameters('data/design_parameters.csv')
MATERIAL_STRENGTHS = load_material_strengths('data/material_strengths.csv')
BEAM_CATALOG = load_beam_catalog('data/beam_catalog.csv')


@dataclass
class CrossSectionGeometry:
    """Calculates geometric properties for different cross-section shapes"""
    A: float  # Area
    Iy: float  # Second moment about y-axis
    Iz: float  # Second moment about z-axis
    J: float   # Torsional constant
    
    @classmethod
    def from_rectangular(cls, base: float, height: float) -> 'CrossSectionGeometry':
        A = base * height
        b, h = min(base, height), max(base, height)
        J = (b**3 * h) * (1/3 - 0.21 * (b/h) * (1 - (b**4)/(12*h**4)))
        Iy = (height * base**3) / 12
        Iz = (base * height**3) / 12
        return cls(A=A, Iy=Iy, Iz=Iz, J=J)
    
    @classmethod
    def from_i_beam(cls, height: float, flange_width: float, flange_thickness: float, web_thickness: float) -> 'CrossSectionGeometry':
        A_flanges = 2 * flange_width * flange_thickness
        A_web = (height - 2 * flange_thickness) * web_thickness
        A = A_flanges + A_web
        
        Iz_flanges = 2 * (flange_width * flange_thickness**3 / 12 + 
                         flange_width * flange_thickness * ((height - flange_thickness)/2)**2)
        web_height = height - 2 * flange_thickness
        Iz_web = web_thickness * web_height**3 / 12
        Iz = Iz_flanges + Iz_web
        
        Iy_flanges = 2 * (flange_thickness * flange_width**3 / 12)
        Iy_web = web_height * web_thickness**3 / 12
        Iy = Iy_flanges + Iy_web
        
        J = (2 * flange_width * flange_thickness**3 + web_height * web_thickness**3) / 3
        return cls(A=A, Iy=Iy, Iz=Iz, J=J)


@dataclass
class BeamSpec:
    catalog_id: str
    beam_type: str  # 'joist', 'tail', 'trimmer', 'header'
    name: str = ''
    quantity: Optional[int] = None
    padding: Optional[float] = None
    
    def __post_init__(self):
        self._catalog_data = BEAM_CATALOG[BEAM_CATALOG['id'] == self.catalog_id].iloc[0]
        self.material = self._catalog_data['material']
        self.base = self._catalog_data['base']
        self.height = self._catalog_data['height']
        self.shape = self._catalog_data['shape']
        self.cost_per_m3 = self._catalog_data['cost_per_m3']
    
    @property
    def section_name(self) -> str:
        return f"sec_{self.catalog_id}"
    
    def get_geometry(self) -> CrossSectionGeometry:
        if self.shape == 'rectangular':
            return CrossSectionGeometry.from_rectangular(self.base, self.height)
        elif self.shape == 'I-beam':
            return CrossSectionGeometry.from_i_beam(self.height, self._catalog_data['flange_width'], self._catalog_data['flange_thickness'], self._catalog_data['web_thickness'])
        else:
            raise ValueError(f"Unknown shape: {self.shape}")
    
    def create_section(self, frame: FEModel3D):
        if self.section_name not in frame.sections:
            geom = self.get_geometry()
            frame.add_section(self.section_name, geom.A, geom.Iy, geom.Iz, geom.J)
    
    def get_volume(self, length: float) -> float:
        geometry = self.get_geometry()
        return geometry.A * length # mm^3
    
    def get_cost(self, length: float) -> float:
        volume_m3 = self.get_volume(length) / 1e9  # Convert mm^3 to m^3
        return volume_m3 * self.cost_per_m3
    
    def copy(self, **kwargs):
        return replace(self, **kwargs)


@dataclass
class BeamPlacement:
    spec: BeamSpec
    x_center: float
    z_start: float = 0
    z_end: float = None

    def add_to_frame(self, frame: FEModel3D, params: DesignParameters, default_z_end: float):
        z_end = self.z_end if self.z_end is not None else default_z_end
        name = self.spec.name

        frame.add_node(Naming.node(name, 'S', floor=True), self.x_center, 0, self.z_start)
        frame.add_node(Naming.node(name, 'S'), self.x_center, params.floor_to_floor, self.z_start)
        
        if self.spec.beam_type != 'tail':
            frame.add_node(Naming.node(name, 'N', floor=True), self.x_center, 0, z_end)
            frame.add_node(Naming.node(name, 'N'), self.x_center, params.floor_to_floor, z_end)
            self.spec.create_section(frame)
            frame.add_member(Naming.member(name), Naming.node(name, 'N'), Naming.node(name, 'S'), self.spec.material, self.spec.section_name)
        else:
            self.spec.create_section(frame)


class GeometryResolver:
    def __init__(
        self,
        east_joists: BeamSpec,
        west_joists: BeamSpec,
        tail_joists: BeamSpec,
        trimmers: BeamSpec,
        header: BeamSpec,
        params: DesignParameters
    ):
        self.east_joists = east_joists
        self.west_joists = west_joists
        self.tail_joists = tail_joists
        self.trimmers = trimmers
        self.header = header
        self.params = params
        self.opening_x_start = params.opening_x_start

        assert self.trimmers.quantity == 2, "Must have exactly 2 trimmers"
        assert self.header.quantity == 1, "Must have exactly 1 header"
        
        # Check padding doesn't exceed available space
        max_east_padding = self.opening_x_start - self.trimmers.base - self.east_joists.base
        assert self.east_joists.padding <= max_east_padding, \
            f"East padding {self.east_joists.padding} exceeds max {max_east_padding}"
        
        opening_x_end = self.opening_x_start + self.params.opening_length
        max_west_padding = self.params.room_length - opening_x_end - self.trimmers.base - self.west_joists.base
        assert self.west_joists.padding <= max_west_padding, \
            f"West padding {self.west_joists.padding} exceeds max {max_west_padding}"

        self.opening_x_end = self.opening_x_start + self.params.opening_length
        
        self.trimmer_E_x_center = self.opening_x_start - (self.trimmers.base / 2)
        self.trimmer_W_x_center = self.opening_x_end + (self.trimmers.base / 2)
        
        self.tail_z_end = self.params.beam_length - self.params.opening_width - self.params.wall_beam_contact_depth/2 - self.header.base/2
        self.header_z_pos = self.tail_z_end
        
        self.opening_z_start = self.params.opening_width + self.params.wall_beam_contact_depth/2

    def _resolve_joist_positions(self, n: int, clear_start: float, clear_end: float, beam_base: float, group: str) -> List[float]:
        def east_positions(n, clear_start, clear_end, beam_base):
            centerline_start = clear_start + beam_base / 2
            centerline_end = clear_end + beam_base / 2
            positions = np.linspace(centerline_start, centerline_end, n+1).tolist()
            return positions[:-1]

        def west_positions(n, clear_start, clear_end, beam_base):
            centerline_start = clear_start - beam_base / 2
            centerline_end = clear_end + beam_base / 2
            positions = np.linspace(centerline_end, centerline_start, n+1).tolist()
            return positions[:-1]

        def tail_positions(n, clear_start, clear_end, beam_base):
            centerline_start = clear_start - beam_base / 2
            centerline_end = clear_end + beam_base / 2
            positions = np.linspace(centerline_start, centerline_end, n+2).tolist()
            return positions[1:-1]

        dispatch = {
            'east': east_positions,
            'west': west_positions,
            'tail': tail_positions
        }

        if group not in dispatch:
            raise ValueError("Invalid group specified")

        return dispatch[group](n, clear_start, clear_end, beam_base)

    def resolve_all_placements(self) -> List[BeamPlacement]:
        all_placements = []
        
        # East joists
        if self.east_joists.quantity > 0:
            joist_spec_east = BeamSpec(
                self.east_joists.catalog_id, 
                'joist',
                quantity=self.east_joists.quantity,
                padding=self.east_joists.padding
            )
            clear_start = self.east_joists.padding
            clear_end = self.trimmer_E_x_center - self.trimmers.base / 2
            east_positions = self._resolve_joist_positions(
                self.east_joists.quantity, 
                clear_start, 
                clear_end, 
                joist_spec_east.base,
                "east"
            )
            for i, x in enumerate(east_positions):
                all_placements.append(BeamPlacement(spec=joist_spec_east.copy(name=f'E{i}'), x_center=x))
        
        # Tail joists
        if self.tail_joists.quantity > 0:
            tail_spec = BeamSpec(
                self.tail_joists.catalog_id, 
                'tail',
                quantity=self.tail_joists.quantity,
                padding=self.tail_joists.padding
            )
            clear_start = self.opening_x_start + self.tail_joists.padding
            clear_end = self.opening_x_end - self.tail_joists.padding
            tail_positions = self._resolve_joist_positions(
                self.tail_joists.quantity, 
                clear_start, 
                clear_end, 
                tail_spec.base,
                "tail"
            )
            for i, x in enumerate(tail_positions):
                all_placements.append(BeamPlacement(spec=tail_spec.copy(name=f'T{i}'), x_center=x, z_end=self.tail_z_end))
        
        # West joists
        if self.west_joists.quantity > 0:
            joist_spec_west = BeamSpec(
                self.west_joists.catalog_id, 
                'joist',
                quantity=self.west_joists.quantity,
                padding=self.west_joists.padding
            )
            clear_start = self.trimmer_W_x_center + self.trimmers.base / 2
            clear_end = self.params.room_length - self.west_joists.padding - self.west_joists.base
            west_positions = self._resolve_joist_positions(
                self.west_joists.quantity, 
                clear_start,
                clear_end,
                joist_spec_west.base,
                "west"
            )
            for i, x in enumerate(west_positions):
                all_placements.append(BeamPlacement(spec=joist_spec_west.copy(name=f'W{i}'), x_center=x))

        # Trimmers
        trimmer_spec = BeamSpec(
            self.trimmers.catalog_id, 
            'trimmer',
            quantity=self.trimmers.quantity,
            padding=self.trimmers.padding
        )
        all_placements.append(BeamPlacement(spec=trimmer_spec.copy(name='trimmer_E'), x_center=self.trimmer_E_x_center))
        all_placements.append(BeamPlacement(spec=trimmer_spec.copy(name='trimmer_W'), x_center=self.trimmer_W_x_center))

        return all_placements

    def add_header_to_frame(self, frame: FEModel3D):
        header_spec = self.header
        header_spec.create_section(frame)
        frame.add_node(Naming.header_node('E'), self.trimmer_E_x_center, 
                       self.params.floor_to_floor, self.header_z_pos)
        frame.add_node(Naming.header_node('W'), self.trimmer_W_x_center, 
                       self.params.floor_to_floor, self.header_z_pos)
        frame.add_member(Naming.header_member(), Naming.header_node('W'), 
                         Naming.header_node('E'), header_spec.material, 
                         header_spec.section_name)

    def connect_tails_to_header(self, frame: FEModel3D, layout: 'LayoutManager'):
        for beam_placement in layout.beams:
            if beam_placement.spec.beam_type == 'tail':
                tail_node_name = Naming.tail_header_node(beam_placement.spec.name)
                frame.add_node(tail_node_name, beam_placement.x_center, self.params.floor_to_floor, self.header_z_pos)
                frame.add_member(
                    Naming.member(beam_placement.spec.name),
                    tail_node_name,
                    Naming.node(beam_placement.spec.name, 'S'),
                    beam_placement.spec.material,
                    beam_placement.spec.section_name
                )


class LayoutManager:
    def __init__(self, params: DesignParameters):
        self.params = params
        self.beams: List[BeamPlacement] = []

    def add_beams(self, placements: List[BeamPlacement]):
        self.beams.extend(placements)

    def get_boundary_beams(self) -> Tuple[BeamPlacement, BeamPlacement]:
        beams_with_position = sorted(self.beams, key=lambda p: p.x_center)
        return beams_with_position[0], beams_with_position[-1]


class LoadApplicator:
    def __init__(self, beams: List[BeamPlacement], params: DesignParameters):
        self.beams = beams
        self.params = params

    def apply_dead_loads(self, frame: FEModel3D):
        for p in self.beams:
            section_geom = p.spec.get_geometry()
            material = MATERIAL_STRENGTHS[p.spec.material]
            dead_load = -section_geom.A * material['rho']
            frame.add_member_dist_load(p.spec.name, 'FY', dead_load, dead_load)
    
    def apply_live_loads(self, frame: FEModel3D):
        sorted_beams = sorted(self.beams, key=lambda p: p.x_center)
        if len(sorted_beams) < 2:
            return

        for i, p in enumerate(sorted_beams):
            # Calculate tributary boundaries
            left_boundary = sorted_beams[i-1].x_center if i > 0 else 0
            right_boundary = sorted_beams[i+1].x_center if i < len(sorted_beams) - 1 else self.params.room_length
            
            trib_left = (p.x_center - left_boundary) / 2
            trib_right = (right_boundary - p.x_center) / 2
            
            load_left = self.params.live_load_mpa * trib_left
            load_right = self.params.live_load_mpa * trib_right

            if p.spec.beam_type in ['joist', 'tail']:
                total_load = load_left + load_right
                frame.add_member_dist_load(p.spec.name, 'FY', total_load, total_load)
            elif p.spec.beam_type == 'trimmer':
                # For trimmers, one side has a load break at the opening
                is_left_opening = i > 0 and sorted_beams[i-1].spec.beam_type == 'tail'
                is_right_opening = i < len(sorted_beams)-1 and sorted_beams[i+1].spec.beam_type == 'tail'
                
                if is_left_opening:
                    frame.add_member_dist_load(p.spec.name, 'FY', load_left, load_left, 0, self.params.opening_width + self.params.wall_beam_contact_depth/2)
                    frame.add_member_dist_load(p.spec.name, 'FY', load_right, load_right)
                elif is_right_opening:
                    frame.add_member_dist_load(p.spec.name, 'FY', load_left, load_left)
                    frame.add_member_dist_load(p.spec.name, 'FY', load_right, load_right, 0, self.params.opening_width + self.params.wall_beam_contact_depth/2)
                else:
                    frame.add_member_dist_load(p.spec.name, 'FY', load_left + load_right, load_left + load_right)


def auto_add_walls(frame, layout, params, wall_thickness, material):
    frame.add_node(Naming.corner_node('SW', floor=True), 0, 0, 0)
    frame.add_node(Naming.corner_node('SW', floor=False), 0, params.floor_to_floor, 0)
    frame.add_node(Naming.corner_node('NW', floor=True), 0, 0, params.beam_length)
    frame.add_node(Naming.corner_node('NW', floor=False), 0, params.floor_to_floor, params.beam_length)
    frame.add_node(Naming.corner_node('SE', floor=True), params.room_length, 0, 0)
    frame.add_node(Naming.corner_node('SE', floor=False), params.room_length, params.floor_to_floor, 0)
    frame.add_node(Naming.corner_node('NE', floor=True), params.room_length, 0, params.beam_length)
    frame.add_node(Naming.corner_node('NE', floor=False), params.room_length, params.floor_to_floor, params.beam_length)

    frame.add_quad(
        Naming.side_wall('west'),
        Naming.corner_node('SW', floor=True),
        Naming.corner_node('NW', floor=True),
        Naming.corner_node('NW', floor=False),
        Naming.corner_node('SW', floor=False),
        wall_thickness, material
    )
    
    frame.add_quad(
        Naming.side_wall('east'),
        Naming.corner_node('SE', floor=True),
        Naming.corner_node('NE', floor=True),
        Naming.corner_node('NE', floor=False),
        Naming.corner_node('SE', floor=False),
        wall_thickness, material
    )
    
    sorted_beams = sorted(layout.beams, key=lambda p: p.x_center)
    if sorted_beams:
        first_beam = sorted_beams[0]
        frame.add_quad(
            Naming.wall('corner_W', first_beam.spec.name, 'south'),
            Naming.corner_node('SW', floor=True),
            Naming.node(first_beam.spec.name, 'S', floor=True),
            Naming.node(first_beam.spec.name, 'S'),
            Naming.corner_node('SW', floor=False),
            wall_thickness, material
        )
        
        for i in range(len(sorted_beams) - 1):
            prev_beam = sorted_beams[i]
            beam = sorted_beams[i+1]
            frame.add_quad(
                Naming.wall(prev_beam.spec.name, beam.spec.name, 'south'),
                Naming.node(prev_beam.spec.name, 'S', floor=True),
                Naming.node(beam.spec.name, 'S', floor=True),
                Naming.node(beam.spec.name, 'S'),
                Naming.node(prev_beam.spec.name, 'S'),
                wall_thickness, material
            )

        last_beam = sorted_beams[-1]
        frame.add_quad(
            Naming.wall(last_beam.spec.name, 'corner_E', 'south'),
            Naming.node(last_beam.spec.name, 'S', floor=True),
            Naming.corner_node('SE', floor=True),
            Naming.corner_node('SE', floor=False),
            Naming.node(last_beam.spec.name, 'S'),
            wall_thickness, material
        )
    
    north_reaching_beams = [b for b in layout.beams if b.spec.beam_type not in ['tail', 'header']]
    north_reaching_beams.sort(key=lambda p: p.x_center)
    if north_reaching_beams:
        first_beam = north_reaching_beams[0]
        frame.add_quad(
            Naming.wall('corner_W', first_beam.spec.name, 'north'),
            Naming.corner_node('NW', floor=True),
            Naming.node(first_beam.spec.name, 'N', floor=True),
            Naming.node(first_beam.spec.name, 'N'),
            Naming.corner_node('NW', floor=False),
            wall_thickness, material
        )

        for i in range(len(north_reaching_beams) - 1):
            prev_beam = north_reaching_beams[i]
            beam = north_reaching_beams[i+1]
            frame.add_quad(
                Naming.wall(prev_beam.spec.name, beam.spec.name, 'north'),
                Naming.node(prev_beam.spec.name, 'N', floor=True),
                Naming.node(beam.spec.name, 'N', floor=True),
                Naming.node(beam.spec.name, 'N'),
                Naming.node(prev_beam.spec.name, 'N'),
                wall_thickness, material
            )

        last_beam = north_reaching_beams[-1]
        frame.add_quad(
            Naming.wall(last_beam.spec.name, 'corner_E', 'north'),
            Naming.node(last_beam.spec.name, 'N', floor=True),
            Naming.corner_node('NE', floor=True),
            Naming.corner_node('NE', floor=False),
            Naming.node(last_beam.spec.name, 'N'),
            wall_thickness, material
        )


@dataclass
class BeamTelemetry:
    name: str
    material: str
    catalog_id: str
    length: float
    max_moment_mz: float
    min_moment_mz: float
    max_shear_fy: float
    min_shear_fy: float
    max_deflection: float
    min_deflection: float
    volume: float
    cost: float
    stress_max: float
    shear_stress_max: float
    passes_bending: bool
    passes_shear: bool

@dataclass
class GroupTelemetry:
    group_name: str
    count: int
    max_deflection: float
    mean_deflection: float
    max_moment: float
    mean_moment: float
    max_shear: float
    mean_shear: float
    total_volume: float
    total_cost: float
    all_pass_bending: bool
    all_pass_shear: bool
    beam_details: List[BeamTelemetry]

@dataclass
class SystemTelemetry:
    max_header_deflection_mm: float
    max_deflection_overall_mm: float
    worst_member: str
    total_volume_m3: float
    total_cost: float
    system_passes: bool

@dataclass
class Telemetry:
    beam_telemetries: Dict[str, BeamTelemetry] = field(default_factory=dict)
    group_telemetries: Dict[str, GroupTelemetry] = field(default_factory=dict)
    system_telemetry: SystemTelemetry = None


class TelemetryCollector:
    """Collects and analyzes telemetry from FE analysis"""
    
    def __init__(self, frame, beam_specs: Dict[str, BeamSpec]):
        self.frame = frame
        self.beam_specs = beam_specs
        
    def calculate_beam_stress(self, member_name: str, moment: float, spec: BeamSpec) -> float:
        """Calculate bending stress: sigma = M * c / I"""
        geometry = spec.get_geometry()
        c = spec.height / 2
        stress = abs(moment * c / geometry.Iz)
        return stress
    
    def calculate_shear_stress(self, member_name: str, shear: float, spec: BeamSpec) -> float:
        """Calculate average shear stress"""
        geometry = spec.get_geometry()
        if spec.shape == 'rectangular':
            tau = 1.5 * abs(shear) / geometry.A
        else:  # I-beam
            catalog_data = spec._catalog_data
            web_area = (spec.height - 2*catalog_data['flange_thickness']) * catalog_data['web_thickness']
            tau = abs(shear) / web_area
        return tau
    
    def collect_beam_telemetry(self, member_name: str, spec: BeamSpec) -> BeamTelemetry:
        member = self.frame.members[member_name]
        
        # Get forces and deflections
        max_moment = member.max_moment('Mz', 'Combo 1')
        min_moment = member.min_moment('Mz', 'Combo 1')
        max_shear = member.max_shear('Fy', 'Combo 1')
        min_shear = member.min_shear('Fy', 'Combo 1')
        max_deflection = member.max_deflection('dy', 'Combo 1')
        min_deflection = member.min_deflection('dy', 'Combo 1')
        
        # Calculate stresses
        moment_for_stress = max(abs(max_moment), abs(min_moment))
        stress_max = self.calculate_beam_stress(member_name, moment_for_stress, spec)
        
        shear_for_stress = max(abs(max_shear), abs(min_shear))
        shear_stress_max = self.calculate_shear_stress(member_name, shear_for_stress, spec)
        
        # Check against material strengths
        material_props = MATERIAL_STRENGTHS[spec.material]
        passes_bending = stress_max <= material_props['f_mk']
        passes_shear = shear_stress_max <= material_props['f_vk']
        
        # Calculate cost and volume
        length = member.L()
        volume = spec.get_volume(length)
        cost = spec.get_cost(length)
        
        return BeamTelemetry(
            name=member_name,
            material=spec.material,
            catalog_id=spec.catalog_id,
            length=length,
            max_moment_mz=max_moment,
            min_moment_mz=min_moment,
            max_shear_fy=max_shear,
            min_shear_fy=min_shear,
            max_deflection=max_deflection,
            min_deflection=min_deflection,
            volume=volume,
            cost=cost,
            stress_max=stress_max,
            shear_stress_max=shear_stress_max,
            passes_bending=passes_bending,
            passes_shear=passes_shear
        )
    
    def collect_group_telemetry(self, group_name: str, member_names: List[str]) -> GroupTelemetry:
        beam_telemetries = [self.collect_beam_telemetry(name, self.beam_specs[name]) for name in member_names]
        
        if not beam_telemetries:
            return GroupTelemetry(
                group_name=group_name, count=0, max_deflection=0, mean_deflection=0,
                max_moment=0, mean_moment=0, max_shear=0, mean_shear=0,
                total_volume=0, total_cost=0, all_pass_bending=True, all_pass_shear=True,
                beam_details=[]
            )
        
        deflections = [abs(b.min_deflection) for b in beam_telemetries]
        moments = [max(abs(b.max_moment_mz), abs(b.min_moment_mz)) for b in beam_telemetries]
        shears = [max(abs(b.max_shear_fy), abs(b.min_shear_fy)) for b in beam_telemetries]
        
        return GroupTelemetry(
            group_name=group_name,
            count=len(beam_telemetries),
            max_deflection=max(deflections),
            mean_deflection=np.mean(deflections),
            max_moment=max(moments),
            mean_moment=np.mean(moments),
            max_shear=max(shears),
            mean_shear=np.mean(shears),
            total_volume=sum(b.volume for b in beam_telemetries),
            total_cost=sum(b.cost for b in beam_telemetries),
            all_pass_bending=all(b.passes_bending for b in beam_telemetries),
            all_pass_shear=all(b.passes_shear for b in beam_telemetries),
            beam_details=beam_telemetries
        )
    
    def collect_system_telemetry(self, beam_groups: Dict[str, List[str]]) -> Telemetry:
        """Collect telemetry for entire system"""
        telemetry = Telemetry()
        
        for group_name, member_names in beam_groups.items():
            if not member_names:
                continue
            group_telem = self.collect_group_telemetry(group_name, member_names)
            telemetry.group_telemetries[group_name] = group_telem
            for beam_telem in group_telem.beam_details:
                telemetry.beam_telemetries[beam_telem.name] = beam_telem
        
        # Overall metrics
        all_beam_stats = list(telemetry.beam_telemetries.values())
        max_deflection_overall = max((abs(b.min_deflection) for b in all_beam_stats), default=0)
        worst_member = max(all_beam_stats, key=lambda b: abs(b.min_deflection), default=None)
        
        total_volume = sum(g.total_volume for g in telemetry.group_telemetries.values())
        total_cost = sum(g.total_cost for g in telemetry.group_telemetries.values())
        
        system_passes = all(g.all_pass_bending and g.all_pass_shear for g in telemetry.group_telemetries.values())
        
        header_deflection = abs(self.frame.members[Naming.header_member()].min_deflection('dy', 'Combo 1'))
        
        telemetry.system_telemetry = SystemTelemetry(
            max_header_deflection_mm=header_deflection,
            max_deflection_overall_mm=max_deflection_overall,
            worst_member=worst_member.name if worst_member else '',
            total_volume_m3=total_volume / 1e9,
            total_cost=total_cost,
            system_passes=system_passes,
        )
        return telemetry

def generate_and_analyze_floor(
    east_joists: BeamSpec,
    west_joists: BeamSpec,
    tail_joists: BeamSpec,
    trimmers: BeamSpec,
    header: BeamSpec,
) -> tuple:
    """Generate floor model, run analysis, and return frame with results"""
    resolver = GeometryResolver(
        east_joists=east_joists,
        west_joists=west_joists,
        tail_joists=tail_joists,
        trimmers=trimmers,
        header=header,
        params=DEFAULT_PARAMS
    )
    all_placements = resolver.resolve_all_placements()
    frame = FEModel3D()
    
    # Add materials
    for mat_name, props in MATERIAL_STRENGTHS.items():
        frame.add_material(
            mat_name, 
            E=props['E'], 
            G=props['E'] / (2 * (1 + props['nu'])), 
            nu=props['nu'], 
            rho=props['rho']
        )
    
    # Build beam spec map for telemetry
    beam_spec_map = {}
    for beam_placement in all_placements:
        beam_spec_map[beam_placement.spec.name] = beam_placement.spec
    
    # Create layout and add all beams
    layout = LayoutManager(params=DEFAULT_PARAMS)
    layout.add_beams(all_placements)
    
    for beam_placement in layout.beams:
        beam_placement.add_to_frame(frame, DEFAULT_PARAMS, DEFAULT_PARAMS.beam_length)

    resolver.add_header_to_frame(frame)
    resolver.connect_tails_to_header(frame, layout)

    # Add header to beam_spec_map for telemetry
    beam_spec_map['header'] = resolver.header
    
    # Add walls and supports
    auto_add_walls(frame, layout, DEFAULT_PARAMS, wall_thickness=DEFAULT_PARAMS.wall_thickness, material='brick')
    for node_name in frame.nodes:
        if node_name.startswith('floor') or 'floor' in node_name:
            frame.def_support(node_name, True, True, True, True, True, True)
    
    # Apply loads
    header_geom = resolver.header.get_geometry()
    header_material = MATERIAL_STRENGTHS[resolver.header.material]
    header_dead_load = -header_geom.A * header_material['rho']
    frame.add_member_dist_load(Naming.header_member(), 'FY', header_dead_load, header_dead_load)

    # load_applicator = LoadApplicator(layout.beams, layout.params, resolver)
    load_applicator = LoadApplicator(layout.beams, layout.params)
    load_applicator.apply_dead_loads(frame)
    load_applicator.apply_live_loads(frame)
    
    # Run analysis
    try:
        frame.analyze(check_statics=True)
    except Exception as e:
        raise RuntimeError(f"Analysis failed: {e}")
    
    return frame, resolver, beam_spec_map


# OPTIMIZATION FRAMEWORK
class FloorOptimizer:
    """Handles optimization and telemetry collection"""
    
    def __init__(self):
        self.results = []
        
    def run_single_configuration(
        self,
        east_joists: BeamSpec,
        west_joists: BeamSpec,
        tail_joists: BeamSpec,
        trimmers: BeamSpec,
        header: BeamSpec,
        config_id: int = None
    ) -> Tuple[Optional[FEModel3D], Dict]:
        """Run analysis for a single configuration and collect telemetry"""
        frame = None
        try:
            frame, resolver, beam_spec_map = generate_and_analyze_floor(
                east_joists=east_joists,
                west_joists=west_joists,
                tail_joists=tail_joists,
                trimmers=trimmers,
                header=header,
            )
            
            # Collect telemetry
            collector = TelemetryCollector(frame, beam_spec_map)
            
            # Define beam groups
            beam_groups = {
                'east_joists': [name for name, spec in beam_spec_map.items() if spec.beam_type == 'joist' and name.startswith('E')],
                'west_joists': [name for name, spec in beam_spec_map.items() if spec.beam_type == 'joist' and name.startswith('W')],
                'tail_joists': [name for name, spec in beam_spec_map.items() if spec.beam_type == 'tail'],
                'trimmers': [name for name, spec in beam_spec_map.items() if spec.beam_type == 'trimmer'],
                'header': [name for name, spec in beam_spec_map.items() if spec.beam_type == 'header']
            }
            
            telemetry = collector.collect_system_telemetry(beam_groups)
            
            return frame, {
                'config_id': config_id,
                'east_joists': east_joists,
                'west_joists': west_joists,
                'tail_joists': tail_joists,
                'trimmers': trimmers,
                'header': header,
                'telemetry': telemetry,
                'success': True,
                'error': None
            }
            
        except Exception as e:
            return None, {
                'config_id': config_id,
                'east_joists': east_joists,
                'west_joists': west_joists,
                'tail_joists': tail_joists,
                'trimmers': trimmers,
                'header': header,
                'telemetry': None,
                'success': False,
                'error': str(e)
            }
    
    def run_grid_search(self, param_space: Dict[str, List]) -> pd.DataFrame:
        """
        Run grid search over parameter space
        
        Args:
            param_space: Dictionary defining parameter ranges, e.g.:
                {
                    'east_joist_catalog_id': ['W60x120', 'W80x160'],
                    'east_quantity': [2, 3, 4],
                    'trimmer_catalog_id': ['W80x160', 'W100x200'],
                    'opening_x_start': [800, 820, 840],
                    ...
                }
        """
        keys = list(param_space.keys())
        combinations = list(product(*[param_space[k] for k in keys]))
        
        print(f"Running grid search over {len(combinations)} configurations...")
        
        for i, combo in tqdm(enumerate(combinations), total=len(combinations)):
            config_params = dict(zip(keys, combo))
            
            # Build configuration
            east_joists_spec = BeamSpec(
                catalog_id=config_params['east_joist_catalog_id'],
                beam_type='joist',
                quantity=config_params['east_quantity'],
                padding=config_params.get('east_padding', 0)
            )
            west_joists_spec = BeamSpec(
                catalog_id=config_params['west_joist_catalog_id'],
                beam_type='joist',
                quantity=config_params['west_quantity'],
                padding=config_params.get('west_padding', 0)
            )
            tail_joists_spec = BeamSpec(
                catalog_id=config_params['tail_joist_catalog_id'],
                beam_type='tail',
                quantity=config_params['tail_quantity'],
                padding=config_params.get('tail_padding', 0)
            )
            trimmers_spec = BeamSpec(
                catalog_id=config_params['trimmer_catalog_id'],
                beam_type='trimmer',
                quantity=2
            )
            header_spec = BeamSpec(
                catalog_id=config_params['header_catalog_id'],
                beam_type='header',
                quantity=1
            )
            
            _, result = self.run_single_configuration(
                east_joists=east_joists_spec,
                west_joists=west_joists_spec,
                tail_joists=tail_joists_spec,
                trimmers=trimmers_spec,
                header=header_spec,
                config_id=i
            )
            self.results.append(result)
            
            if (i + 1) % 10 == 0:
                print(f"Completed {i + 1}/{len(combinations)} configurations")
        
        return self.results_to_dataframe()
    
    def results_to_dataframe(self) -> pd.DataFrame:
        """Convert results to pandas DataFrame for analysis"""
        rows = []
        
        for result in self.results:
            if not result['success']:
                row = {
                    'config_id': result['config_id'],
                    'success': False,
                    'error': result['error']
                }
                rows.append(row)
                continue
            
            telem = result['telemetry']
            
            row = {
                # Configuration
                'config_id': result['config_id'],
                'east_catalog_id': result['east_joists'].catalog_id,
                'east_quantity': result['east_joists'].quantity,
                'west_catalog_id': result['west_joists'].catalog_id,
                'west_quantity': result['west_joists'].quantity,
                'tail_catalog_id': result['tail_joists'].catalog_id,
                'tail_quantity': result['tail_joists'].quantity,
                'trimmer_catalog_id': result['trimmers'].catalog_id,
                'header_catalog_id': result['header'].catalog_id,
                'opening_x_start': DEFAULT_PARAMS.opening_x_start,
                
                # Overall metrics
                'max_deflection_mm': telem.system_telemetry.max_deflection_overall_mm,
                'header_deflection_mm': telem.system_telemetry.max_header_deflection_mm,
                'worst_member': telem.system_telemetry.worst_member,
                'total_volume_m3': telem.system_telemetry.total_volume_m3,
                'total_cost': telem.system_telemetry.total_cost,
                'system_passes': telem.system_telemetry.system_passes,
                
                # Group-level metrics
                **{f'{group}_max_deflection': stats.max_deflection 
                   for group, stats in telem.group_telemetries.items()},
                **{f'{group}_mean_deflection': stats.mean_deflection 
                   for group, stats in telem.group_telemetries.items()},
                **{f'{group}_max_moment': stats.max_moment 
                   for group, stats in telem.group_telemetries.items()},
                **{f'{group}_passes_bending': stats.all_pass_bending 
                   for group, stats in telem.group_telemetries.items()},
                **{f'{group}_passes_shear': stats.all_pass_shear 
                   for group, stats in telem.group_telemetries.items()},
                **{f'{group}_total_cost': stats.total_cost 
                   for group, stats in telem.group_telemetries.items()},
                
                'success': True,
                'error': None
            }
            
            rows.append(row)
        
        return pd.DataFrame(rows)
    
    def get_pareto_front(self, df: pd.DataFrame, objectives: List[str] = ['total_cost', 'max_deflection_mm']) -> pd.DataFrame:
        """Find Pareto-optimal solutions"""
        valid = df[(df['success'] == True) & (df['system_passes'] == True)].copy()
        
        if len(valid) == 0:
            print("No valid configurations found!")
            return pd.DataFrame()
        
        # Find Pareto front
        is_pareto = np.ones(len(valid), dtype=bool)
        
        for i, row_i in valid.iterrows():
            for j, row_j in valid.iterrows():
                if i == j:
                    continue
                
                # Check if j dominates i
                dominates = True
                for obj in objectives:
                    if row_j[obj] > row_i[obj]:
                        dominates = False
                        break
                
                if dominates:
                    strictly_better = any(row_j[obj] < row_i[obj] for obj in objectives)
                    if strictly_better:
                        is_pareto[valid.index.get_loc(i)] = False
                        break
        
        pareto_indices = valid.index[is_pareto]
        return valid.loc[pareto_indices]
    
if __name__ == '__main__':
    # Single solution with rendering
    optimizer = FloorOptimizer()
    frame, result = optimizer.run_single_configuration(
        east_joists=BeamSpec(catalog_id='W60x120', beam_type='joist', quantity=1, padding=0),
        west_joists=BeamSpec(catalog_id='W60x120', beam_type='joist', quantity=1, padding=0),
        tail_joists=BeamSpec(catalog_id='W60x120', beam_type='tail', quantity=1, padding=0),
        trimmers=BeamSpec(catalog_id='W80x160', beam_type='trimmer', quantity=2),
        header=BeamSpec(catalog_id='W60x120', beam_type='header', quantity=1),
        config_id=0
    )

    if result['success']:
        telem = result['telemetry']
        print(f"\nSystem passes all checks: {telem.system_telemetry.system_passes}")
        print(f"Total cost: ${telem.system_telemetry.total_cost:.2f}")
        print(f"Total volume: {telem.system_telemetry.total_volume_m3:.4f} m³")
        print(f"Max deflection: {telem.system_telemetry.max_deflection_overall_mm:.3f} mm")
        print(f"Header deflection: {telem.system_telemetry.max_header_deflection_mm:.3f} mm")
        print(f"Worst member: {telem.system_telemetry.worst_member}")
        
        print("\nGroup Statistics:")
        for group_name, stats in telem.group_telemetries.items():
            print(f"\n  {group_name}:")
            print(f"    Count: {stats.count}")
            print(f"    Max deflection: {stats.max_deflection:.3f} mm")
            print(f"    Mean deflection: {stats.mean_deflection:.3f} mm")
            print(f"    Cost: ${stats.total_cost:.2f}")
            print(f"    Passes bending: {stats.all_pass_bending}")
            print(f"    Passes shear: {stats.all_pass_shear}")
    else:
        print(f"Analysis failed: {result['error']}")


    def set_wall_opacity(plotter, opacity=0.5):
        for actor in plotter.renderer.actors.values():
            if (hasattr(actor, 'mapper') and
                hasattr(actor.mapper, 'dataset') and
                actor.mapper.dataset.n_faces_strict > 0):
                actor.prop.opacity = opacity

    rndr = Renderer(frame)
    rndr.annotation_size = 5
    rndr.render_loads = False
    rndr.deformed_shape = True
    rndr.deformed_scale = 1000
    opacity = 0.25
    rndr.post_update_callbacks.append(lambda plotter: set_wall_opacity(plotter, opacity=opacity))
    rndr.render_model()

In [ ]:
(515/8)


In [ ]:
from dataclasses import dataclass, replace
from re import L
from typing import List, Dict, Optional, Tuple, Literal
from Pynite import FEModel3D
from Pynite.Rendering import Renderer
import numpy as np
import pandas as pd
from collections import defaultdict
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from skopt.utils import use_named_args
import warnings
import bisect
import time


@dataclass
class DesignParameters:
    room_length: float
    room_width: float
    room_height: float
    plank_thickness: float
    opening_width: float
    opening_length: float
    opening_x_start: float
    wall_beam_contact_depth: float
    live_load_mpa: float
    wall_thickness: float

    @property
    def floor_y(self):
        return self.room_height / 2

    @property
    def beam_length(self):
        return self.room_width + self.wall_beam_contact_depth
    
    @property
    def opening_z(self):
        return self.beam_length - self.opening_width - self.wall_beam_contact_depth/2


def prep_data():
    # Units are mm, N, and MPa (N/mm²)
    input_params = pd.read_csv('data/design_parameters.csv').iloc[0].to_dict()
    input_params = DesignParameters(**input_params)

    material_str = pd.read_csv('data/material_strengths.csv').set_index('material').to_dict(orient='index')

    material_catalog = pd.read_csv('data/material_catalog.csv')
    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    # Add double beams
    doubled_beams = material_catalog[(material_catalog['type'] == 'beam') & (material_catalog['material'] == 'c24')].copy()
    doubled_beams['base'] = doubled_beams['base'] * 2
    doubled_beams['type'] = 'double'
    doubled_beams['source'] = doubled_beams['id']
    material_catalog = pd.concat([material_catalog, doubled_beams], ignore_index=True)

    # Standardize floor materials to 200mm width
    is_floor = material_catalog['type'] == 'floor'
    material_catalog.loc[is_floor, 'cost_unit'] = material_catalog.loc[is_floor, 'cost_unit'] * (200 / material_catalog.loc[is_floor, 'base'])
    material_catalog.loc[is_floor, 'base'] = 200

    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    return input_params, material_str, material_catalog

@dataclass
class CrossSectionProperties:
    A: float  # Area
    Iy: float  # Second moment about y-axis
    Iz: float  # Second moment about z-axis
    J: float   # Torsional constant
    
    @classmethod
    def from_rectangular(cls, base: float, height: float) -> 'CrossSectionProperties':
        A = base * height
        b, h = min(base, height), max(base, height)
        J = (b**3 * h) * (1/3 - 0.21 * (b/h) * (1 - (b**4)/(12*h**4)))
        Iy = (height * base**3) / 12
        Iz = (base * height**3) / 12
        return cls(A=A, Iy=Iy, Iz=Iz, J=J)
    
    @classmethod
    def from_i_beam(cls, height: float, flange_width: float, flange_thickness: float, web_thickness: float) -> 'CrossSectionProperties':
        A_flanges = 2 * flange_width * flange_thickness
        A_web = (height - 2 * flange_thickness) * web_thickness
        A = A_flanges + A_web
        
        Iz_flanges = 2 * (flange_width * flange_thickness**3 / 12 + 
                         flange_width * flange_thickness * ((height - flange_thickness)/2)**2)
        web_height = height - 2 * flange_thickness
        Iz_web = web_thickness * web_height**3 / 12
        Iz = Iz_flanges + Iz_web
        
        Iy_flanges = 2 * (flange_thickness * flange_width**3 / 12)
        Iy_web = web_height * web_thickness**3 / 12
        Iy = Iy_flanges + Iy_web
        
        J = (2 * flange_width * flange_thickness**3 + web_height * web_thickness**3) / 3
        return cls(A=A, Iy=Iy, Iz=Iz, J=J)

@dataclass
class MemberSpec:
    material_id: str
    quantity: Optional[int] = None
    padding: Optional[float] = None
    
    def __post_init__(self):
        self._catalog_data = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == self.material_id].iloc[0]
        self.material = self._catalog_data['material']
        self.type = self._catalog_data['type']
        self.base = self._catalog_data['base']
        self.height = self._catalog_data['height']
        self.length = self._catalog_data['length']
        self.shape = self._catalog_data['shape']
        self.cost_per_m3 = self._catalog_data['cost_unit']
    
    @property
    def section_name(self) -> str:
        return f"sec_{self.material_id}"
    
    def get_geometry(self) -> CrossSectionProperties:
        if self.shape == 'rectangular':
            return CrossSectionProperties.from_rectangular(self.base, self.height)
        elif self.shape == 'I-beam':
            return CrossSectionProperties.from_i_beam(self.height, self._catalog_data['flange_width'], self._catalog_data['flange_thickness'], self._catalog_data['web_thickness'])
        else:
            raise ValueError(f"Unknown shape: {self.shape}")
    
    def create_section(self, frame: FEModel3D):
        if self.section_name not in frame.sections:
            geom = self.get_geometry()
            frame.add_section(self.section_name, geom.A, geom.Iy, geom.Iz, geom.J)

@dataclass
class NodeLocation:
    name: str
    X: float
    Y: float
    Z: float

@dataclass
class Member:
    name: str
    node_i: str
    node_j: str
    spec: MemberSpec
    

def calculate_nodes_and_members(
    east_joists: MemberSpec,
    west_joists: MemberSpec,
    tail_joists: MemberSpec,
    trimmers: MemberSpec,
    header: MemberSpec,
    planks: MemberSpec) -> Tuple[List[NodeLocation], List[Member]]:

    def _calculate_evenly_spaced_positions(
        n: int, 
        clear_start: float, 
        clear_end: float, 
        member_base: float,
        distribution: Literal['start_aligned', 'end_aligned', 'centered']) -> List[float]:

        if n == 0:
            return []
        
        if distribution == 'start_aligned':
            centerline_start = clear_start + member_base / 2
            centerline_end = clear_end + member_base / 2
            positions = np.linspace(centerline_start, centerline_end, n + 1).tolist()
            return positions[:-1]
        
        elif distribution == 'end_aligned':
            centerline_start = clear_start - member_base / 2
            centerline_end = clear_end - member_base / 2
            positions = np.linspace(centerline_start, centerline_end, n + 1).tolist()
            return positions[1:]
        
        elif distribution == 'centered':
            centerline_start = clear_start + member_base / 2
            centerline_end = clear_end - member_base / 2
            positions = np.linspace(centerline_start, centerline_end, n + 2).tolist()
            return positions[1:-1]
        else:
            raise ValueError(f"Unknown distribution: {distribution}")

    assert trimmers.quantity == 2, "Must have exactly 2 trimmers"
    assert header.quantity == 1, "Must have exactly 1 header"
    
    # Stairs opening params
    opening_x_end = INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length
    trimmer_E_x = INPUT_PARAMS.opening_x_start - (trimmers.base / 2)
    trimmer_W_x = opening_x_end + (trimmers.base / 2)
    header_z = INPUT_PARAMS.opening_z - header.base/2
    
    # Beam centerline positions
    beam_positions = {}
    if east_joists.quantity > 0:
        clear_start = east_joists.padding
        clear_end = trimmer_E_x - trimmers.base / 2
        x_positions = _calculate_evenly_spaced_positions(east_joists.quantity, clear_start, clear_end, east_joists.base, 'start_aligned')
        beam_positions['east'] = [(f'east{i}', x) for i, x in enumerate(x_positions)]
    
    if tail_joists.quantity > 0:
        clear_start = INPUT_PARAMS.opening_x_start + tail_joists.padding
        clear_end = opening_x_end - tail_joists.padding
        x_positions = _calculate_evenly_spaced_positions(tail_joists.quantity, clear_start, clear_end, tail_joists.base, 'centered')
        beam_positions['tail'] = [(f'tail{i}', x) for i, x in enumerate(x_positions)]
    
    if west_joists.quantity > 0:
        clear_start = trimmer_W_x + trimmers.base / 2
        clear_end = INPUT_PARAMS.room_length - west_joists.padding
        x_positions = _calculate_evenly_spaced_positions(west_joists.quantity, clear_start, clear_end, west_joists.base, 'end_aligned')
        beam_positions['west'] = [(f'west{i}', x) for i, x in enumerate(x_positions)]
    
    beam_positions['trimmer'] = [('trimmerE', trimmer_E_x), ('trimmerW', trimmer_W_x)]

    # Plank centerline positions
    clear_start = INPUT_PARAMS.wall_beam_contact_depth / 2
    clear_end = INPUT_PARAMS.beam_length - INPUT_PARAMS.wall_beam_contact_depth / 2
    planks.quantity = int((clear_end - clear_start) // planks.base)
    z_positions = _calculate_evenly_spaced_positions(planks.quantity, clear_start-planks.base, clear_end+planks.base, planks.base, 'centered')
    plank_positions = [(f'p{i}', z) for i, z in enumerate(z_positions)]
    
    # Beam nodes and member locations
    nodes = []
    members = []
    for group_name, group_positions in beam_positions.items():
        for beam_name, x in group_positions:
            nodes.append(NodeLocation(f'{beam_name}_S', x, INPUT_PARAMS.floor_y, 0))
            if group_name == 'tail': # Tails connect to header (header_z), not to wall (beam_length)
                nodes.append(NodeLocation(f'{beam_name}_header', x, INPUT_PARAMS.floor_y, header_z))
                members.append(Member(name=beam_name, node_i=f'{beam_name}_header', node_j=f'{beam_name}_S', spec=east_joists))
            else:
                nodes.append(NodeLocation(f'{beam_name}_N', x, INPUT_PARAMS.floor_y, INPUT_PARAMS.beam_length))
                members.append(Member(name=beam_name, node_i=f'{beam_name}_N', node_j=f'{beam_name}_S', spec=east_joists))
    
    nodes.append(NodeLocation('headerE', trimmer_E_x, INPUT_PARAMS.floor_y, header_z))
    nodes.append(NodeLocation('headerW', trimmer_W_x, INPUT_PARAMS.floor_y, header_z))
    members.append(Member(name='header', node_i='headerW', node_j='headerE', spec=header))

    # Corner nodes
    walls = [('E', 0), ('W', INPUT_PARAMS.room_length)]
    for corner_name, x in walls:
        nodes.append(NodeLocation(f'{corner_name}_S', x, INPUT_PARAMS.floor_y, 0))
        nodes.append(NodeLocation(f'{corner_name}_N', x, INPUT_PARAMS.floor_y, INPUT_PARAMS.beam_length))

    # Plank nodes and member locations
    plank_series_dict = {}
    beam_positions['walls'] = walls # So planks extend to walls
    for plank_name, z in plank_positions:
        plank_nodes = []
        for group_positions in beam_positions.values():
            for beam_name, x in group_positions:
                if x > trimmer_E_x and x < trimmer_W_x and z > header_z:
                    continue
                intersection_node = NodeLocation(f'{beam_name}-{plank_name}', x, INPUT_PARAMS.floor_y, z)
                plank_nodes.append(intersection_node)
                nodes.append(intersection_node)
        plank_series_dict[plank_name] = plank_nodes
    
    for plank_name, plank_nodes in plank_series_dict.items():
        plank_nodes.sort(key=lambda node: node.X)
        for i, _ in enumerate(plank_nodes[:-1]):
            if plank_nodes[i].X == trimmer_E_x and plank_nodes[i].Z > header_z:
                continue
            members.append(Member(name=f'{plank_name}_{i}', node_i=plank_nodes[i].name, node_j=plank_nodes[i+1].name, spec=planks))
    
    return nodes, members


def assemble_frame(nodes: List[NodeLocation], members: List[Member]) -> Tuple[FEModel3D, Dict[str, MemberSpec]]:
    frame = FEModel3D()
    for mat_name, props in MATERIAL_STRENGTHS.items():
        if props['G'] is None:
            G = props['E_0'] / (2 * (1 + props['nu']))
        else:
            G = props['G']
        frame.add_material(
            mat_name,
            E=props['E_0'],
            G=G,
            nu=props['nu'],
            rho=props['rho']
        )
    
    for node in nodes:
        frame.add_node(node.name, node.X, node.Y, node.Z)
    
    for member in members:
        member.spec.create_section(frame)
        frame.add_member(
            member.name,
            member.node_i,
            member.node_j,
            member.spec.material,
            member.spec.section_name
        )
    
    return frame

def define_supports(frame, nodes, wall_thickness, material, walls=False) -> None:
    supports = {}
    supports['north'] = sorted([n for n in nodes if n.name.endswith('_N')], key=lambda n: n.X)
    supports['south'] = sorted([n for n in nodes if n.name.endswith('_S')], key=lambda n: n.X)
    supports['east'] = sorted([n for n in nodes if n.name.startswith('E_')], key=lambda n: n.Z)
    supports['west'] = sorted([n for n in nodes if n.name.startswith('W_')], key=lambda n: n.Z)

    if walls:
        floored = []
        for support_side, support_nodes in supports.items():
            for node in support_nodes:
                if node.name not in floored:
                    floored.append(node.name)
                    frame.add_node(f'{node.name}_floor', node.X, 0, node.Z)

        for support_side, support_nodes in supports.items():
            for i, _ in enumerate(support_nodes[:-1]):
                frame.add_quad(
                    f'{support_side}_wall{i}',
                    support_nodes[i].name,
                    f'{support_nodes[i].name}_floor',
                    f'{support_nodes[i+1].name}_floor',
                    support_nodes[i+1].name,
                    wall_thickness, 
                    material
                )
        for node_name in frame.nodes:
            if node_name.endswith('_floor'):
                frame.def_support(node_name, True, True, True, True, True, True)
    else:
        for support_side, support_nodes in supports.items():
            if support_side in ['north', 'south']:
                for node in support_nodes:
                    frame.def_support(node.name, True, True, True, True, True, True)


def create_model(
    east_joists: MemberSpec,
    west_joists: MemberSpec,
    tail_joists: MemberSpec,
    trimmers: MemberSpec,
    header: MemberSpec,
    planks: MemberSpec,
    walls: bool = True) -> Tuple:
    
    nodes, members = calculate_nodes_and_members(east_joists, west_joists, tail_joists, trimmers, header, planks)
    frame = assemble_frame(nodes, members)
    define_supports(frame, nodes, INPUT_PARAMS.wall_thickness, 'brick', walls=walls)
    apply_loads(frame, members)
    
    return frame, nodes, members


def apply_loads(frame: FEModel3D, members: List[Member]) -> None:
    # Dead loads
    for member in members:
        geom = member.spec.get_geometry()
        material = MATERIAL_STRENGTHS[member.spec.material]
        dead_load = -geom.A * material['rho']
        frame.add_member_dist_load(member.name, 'FY', dead_load, dead_load)

    # Live loads
    plank_members = [m for m in members if m.name.startswith('p')]
    plank_z_values = sorted(set(frame.nodes[m.node_i].Z for m in plank_members))
    standard_spacing = plank_z_values[1] - plank_z_values[0]
    
    tail_planks = []
    for m in plank_members:
        if 'tail' in m.node_i or 'tail' in m.node_j:
            tail_planks.append(m)
    
    min_z = min(plank_z_values)
    max_z = max(plank_z_values)
    max_tail_plank_z = max(frame.nodes[m.node_i].Z for m in tail_planks)
    for member in plank_members:
        member_z = frame.nodes[member.node_i].Z
        
        if member_z == min_z:
            tributary_width = member.spec.base / 2 + standard_spacing / 2
        elif member_z == max_z:
            tributary_width = member.spec.base / 2 + standard_spacing / 2
        elif member_z == max_tail_plank_z and member in tail_planks:
            tributary_width = INPUT_PARAMS.opening_z - member_z + standard_spacing / 2
        else:
            tributary_width = standard_spacing
        
        live_load = -INPUT_PARAMS.live_load_mpa * tributary_width
        frame.add_member_dist_load(member.name, 'FY', live_load, live_load)


def calculate_purchase_quantity(frame: FEModel3D, members: List[Member]):

    def _find_optimal_cuts(member_lengths: List[float], stock_length: float):
        member_lengths.sort(reverse=True)
        stock_parts = []
        remaining_lengths_in_stock_parts = []
        for member_len in member_lengths:
            index_for_best_stock_part = bisect.bisect_left(remaining_lengths_in_stock_parts, member_len)
            
            if index_for_best_stock_part < len(remaining_lengths_in_stock_parts):
                old_remaining_lengths = remaining_lengths_in_stock_parts.pop(index_for_best_stock_part)
                best_stock_part = stock_parts.pop(index_for_best_stock_part)
                best_stock_part.append(member_len)

                new_remaining_length = old_remaining_lengths - member_len
                
                new_index_for_stock_part = bisect.bisect_left(remaining_lengths_in_stock_parts, new_remaining_length)
                remaining_lengths_in_stock_parts.insert(new_index_for_stock_part, new_remaining_length)
                stock_parts.insert(new_index_for_stock_part, best_stock_part)
                
            else:
                new_remaining_length = stock_length - member_len
                
                new_insertion_index = bisect.bisect_left(remaining_lengths_in_stock_parts, new_remaining_length)
                remaining_lengths_in_stock_parts.insert(new_insertion_index, new_remaining_length)
                stock_parts.insert(new_insertion_index, [member_len])

        return len(stock_parts), stock_parts
    
    materials = defaultdict(list)
    for member in members:
        pynite_member = frame.members[member.name]
        member_length = float(pynite_member.L())

        if member.spec.type == 'double':
            source_id = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == member.spec.material_id].iloc[0]['source']
            materials[source_id].extend([member_length] * 2)
        else:
            material_id = member.spec.material_id
            materials[material_id].extend([member_length])


    total_cost = 0.0
    all_material_cuts = {}
    for material_id, lengths in materials.items():
        material_specs = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == material_id].iloc[0]
        num_beams_to_buy, current_material_cuts = _find_optimal_cuts(lengths, material_specs['length'])
        all_material_cuts[material_id] = current_material_cuts
        total_cost += num_beams_to_buy * material_specs['cost_unit']
    
    return total_cost, all_material_cuts


def evaluate_stresses(frame: FEModel3D, members: List[Member]) -> List[Dict]:
    frame.analyze(check_statics=True)

    evaluations = []
    for member in members:
        pynite_member = frame.members[member.name]
        geom = member.spec.get_geometry()
        mat_props = MATERIAL_STRENGTHS[member.spec.material]

        # Deflection
        deflection = abs(pynite_member.min_deflection('dy', 'Combo 1'))

        # Bending Stress
        max_moment = max(
            abs(pynite_member.max_moment('Mz', 'Combo 1')),
            abs(pynite_member.min_moment('Mz', 'Combo 1'))
            )
        c = member.spec.height / 2
        bending_stress = (max_moment * c / geom.Iz) if geom.Iz > 0 else 0
        bending_ratio = bending_stress / mat_props['f_mk'] if mat_props['f_mk'] > 0 else 0

        # Shear Stress
        max_shear = max(
            abs(pynite_member.max_shear('Fy', 'Combo 1')),
            abs(pynite_member.min_shear('Fy', 'Combo 1'))
            )
        shear_stress = 0
        if geom.A > 0:
            if member.spec.shape == 'rectangular':
                shear_stress = 1.5 * abs(max_shear) / geom.A
            else:
                catalog_data = member.spec._catalog_data
                web_height = member.spec.height - 2 * catalog_data['flange_thickness']
                web_thickness = catalog_data['web_thickness']
                web_area = web_height * web_thickness
                shear_stress = abs(max_shear) / web_area if web_area > 0 else 0
        shear_ratio = shear_stress / mat_props['f_vk'] if mat_props['f_vk'] > 0 else 0

        passes_strength = (bending_ratio <= 1.0 and shear_ratio <= 1.0)

        evaluations.append({
            'member_name': member.name,
            'material_id': member.spec.material_id,
            'deflection_mm': deflection,
            'bending_stress_ratio': bending_ratio,
            'shear_stress_ratio': shear_ratio,
            'passes_strength_check': passes_strength,
        })

    return evaluations


def render(frame, deformed_scale=100, opacity=0.25) -> None:
    def _set_wall_opacity(plotter, opacity=0.25):
        for actor in plotter.renderer.actors.values():
            if (hasattr(actor, 'mapper') and
                hasattr(actor.mapper, 'dataset') and
                actor.mapper.dataset.n_faces_strict > 0):
                actor.prop.opacity = opacity

    rndr = Renderer(frame)
    rndr.annotation_size = 5
    rndr.render_loads = False
    rndr.deformed_shape = True
    rndr.deformed_scale = deformed_scale
    rndr.post_update_callbacks.append(lambda plotter: _set_wall_opacity(plotter, opacity=opacity))
    rndr.render_model()


if __name__ == '__main__':

    INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG = prep_data()

    east_joists = MemberSpec('c24_60x120', quantity=1, padding=0)
    tail_joists = MemberSpec('c24_60x120', quantity=1, padding=0)
    west_joists = MemberSpec('c24_60x120', quantity=1, padding=0)
    trimmers = MemberSpec('c24_80x160', quantity=2)
    header = MemberSpec('c24_60x120', quantity=1)
    planks = MemberSpec('c18_200x25')
    
    frame, nodes, members = create_model(
        east_joists=east_joists,
        west_joists=west_joists,
        tail_joists=tail_joists,
        trimmers=trimmers,
        header=header,
        planks=planks,
    )

    member_evaluations = evaluate_stresses(frame, members)
    # display(pd.DataFrame(member_evaluations))
    start = time.time()
    total_cost, cuts = calculate_purchase_quantity(frame, members)
    # render(frame, deformed_scale=100, opacity=0.25)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
import pandas as pd

# Units are mm, N, and MPa (N/mm²)
# INPUT PARAMS
input_params = pd.read_csv('data/design_parameters.csv').iloc[0].to_dict()
input_params = DesignParameters(**input_params)

# MATERIAL STRENGTH
material_str = pd.read_csv('data/material_strengths.csv').set_index('material').to_dict(orient='index')

# CONNECTORS
connectors = pd.read_csv('data/connectors.csv')
train_data = connectors.dropna(subset=['grams'])
predict_data = connectors[connectors['grams'].isnull()]

## Fill missing weights
X_train = train_data[['base', 'height']]
y_train = train_data['grams']
X_predict = predict_data[['base', 'height']]
model = LinearRegression()
model.fit(X_train, y_train)
predict_data['grams'] = model.predict(X_predict)
connectors = pd.concat([train_data, predict_data])

## Smoothen price and group by dimensions
X = connectors[['base', 'height']]
y = connectors['price_unit']
huber = HuberRegressor(epsilon=1.24)
huber.fit(X, y)
connectors['price_unit'] = huber.predict(X)
connectors['weight_N'] = connectors['grams'] / 100
connectors = connectors[['base', 'height', 'weight_N', 'price_unit']].groupby(['base', 'height']).mean()
connectors = connectors.reset_index()

# MATERIAL_CATALOG
material_catalog = pd.read_csv('data/material_catalog.csv')
material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

## Add double beams
doubled_beams = material_catalog[(material_catalog['type'] == 'beam') & (material_catalog['material'] != 'steel')].copy()
doubled_beams['base'] = doubled_beams['base'] * 2
doubled_beams['type'] = 'double'
doubled_beams['source'] = doubled_beams['id']
material_catalog = pd.concat([material_catalog, doubled_beams], ignore_index=True)

rows_to_drop = ((material_catalog['type'] == 'beam') | (material_catalog['type'] == 'double')) & (~material_catalog['base'].isin(connectors['base']))
material_catalog = material_catalog[~rows_to_drop]

## Standardize floor materials to 200mm width
is_floor = material_catalog['type'] == 'floor'
material_catalog.loc[is_floor, 'cost_unit'] = material_catalog.loc[is_floor, 'cost_unit'] * (200 / material_catalog.loc[is_floor, 'base'])
material_catalog.loc[is_floor, 'base'] = 200

material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))
material_catalog

In [ ]:
member_connectors = connectors[connectors['base'] == 40]
connector = member_connectors[member_connectors['height'] <= 100]
connector
# if len(connector) > 1:
#     return connector.mean()
# else:
#     return connector
    

In [ ]:
[m for m in frame.members if m.startswith('header')][0]

In [ ]:
# Optimization
def create_optimization_objective(
    objective_weights: Dict[str, float] = None,
    max_deflection_limit: float = 10.0):
    
    if objective_weights is None:
        objective_weights = {'cost': 0.6, 'deflection': 0.3}
    total_weight = sum(objective_weights.values())
    objective_weights = {k: v/total_weight for k, v in objective_weights.items()}
    
    def objective(params_dict: Dict) -> float:
        try:
            east_joists = MemberSpec(params_dict['east_material'], quantity=params_dict['east_quantity'], padding=params_dict['east_padding'])
            west_joists = MemberSpec(params_dict['west_material'], quantity=params_dict['west_quantity'], padding=params_dict['west_padding'])
            tail_joists = MemberSpec(params_dict['tail_material'], quantity=params_dict['tail_quantity'], padding=params_dict['tail_padding'])
            trimmers = MemberSpec(params_dict['trimmer_material'], quantity=2)
            header = MemberSpec(params_dict['header_material'], quantity=1)
            planks = MemberSpec(params_dict['plank_material'])
            
            frame, _, members = create_model(
                east_joists=east_joists,
                west_joists=west_joists,
                tail_joists=tail_joists,
                trimmers=trimmers,
                header=header,
                planks=planks
            )
            total_cost = calculate_purchase_quantity(frame, members)
            frame.analyze(check_statics=True)
            
            results = []
            for member in members:
                pynite_member = frame.members[member.name]
                geom = member.spec.get_geometry()
                strengths = MATERIAL_STRENGTHS[member.spec.material]

                # Bending Stress heck
                # Get max moment. For a floor joist, this is typically about the strong axis (Z-axis in Pynite)
                max_moment = max(abs(pynite_member.max_moment('Mz')), abs(pynite_member.min_moment('Mz')))
                section_modulus_z = geom.Iz / (member.spec.height / 2)
                
                if section_modulus_z > 0:
                    bending_stress = max_moment / section_modulus_z
                else:
                    bending_stress = 0
                    
                bending_strength = strengths['f_mk']
                bending_utilization = bending_stress / bending_strength

                # Shear Stress Check
                # Get max shear force (typically FY for gravity loads)
                max_shear = max(abs(pynite_member.max_shear('Fy')), abs(pynite_member.min_shear('Fy')))

                # Max shear stress for a rectangular section
                if geom.A > 0 and member.spec.shape == 'rectangular':
                    shear_stress = 1.5 * (max_shear / geom.A)
                else:
                    # Note: Shear stress calculation for I-beams is more complex, often dominated by the web.
                    # This is a simplification. A more detailed analysis would be needed for I-beams.
                    shear_stress = max_shear / geom.A 

                shear_strength = strengths['f_vk']
                shear_utilization = shear_stress / shear_strength if shear_strength > 0 else 0
                
                # Deflection Check
                # Pynite calculates nodal displacements. Getting local member deflection is more involved.
                # A simple proxy is to check the max vertical displacement of the member's nodes.
                node_i_disp = frame.nodes[member.node_i].DY['Combo 1']
                node_j_disp = frame.nodes[member.node_j].DY['Combo 1']
                max_deflection = min(node_i_disp, node_j_disp)

                results.append({
                    'Member': member.name,
                    'Type': member.spec.material_id,
                    'Max Moment (N-mm)': max_moment,
                    'Bending Stress (MPa)': bending_stress,
                    'Bending Strength (MPa)': bending_strength,
                    'Bending Utilization': f"{bending_utilization:.2%}",
                    'Max Shear (N)': max_shear,
                    'Shear Stress (MPa)': shear_stress,
                    'Shear Strength (MPa)': shear_strength,
                    'Shear Utilization': f"{shear_utilization:.2%}",
                    'Max Deflection (mm)': max_deflection
                })

            results_df = pd.DataFrame(results)

            # Identify failing members
            failing_bending = results_df[results_df['Bending Utilization'].str.rstrip('%').astype(float) > 100]
            failing_shear = results_df[results_df['Shear Utilization'].str.rstrip('%').astype(float) > 100]

            if not failing_bending.empty:
                print(failing_bending[['Member', 'Type', 'Bending Utilization']])
            if not failing_shear.empty:
                print(failing_shear[['Member', 'Type', 'Shear Utilization']])



            max_deflection = 0
            max_stress_ratio = 0
            for member_name, pynite_member in frame.members.items():
                
                # Deflection
                deflection = abs(member.min_deflection('dy', 'Combo 1'))
                max_deflection = max(max_deflection, deflection)
                
                # Stress check
                max_moment = max(
                    abs(member.max_moment('Mz', 'Combo 1')),
                    abs(member.min_moment('Mz', 'Combo 1'))
                )
                max_shear = max(
                    abs(member.max_shear('Fy', 'Combo 1')),
                    abs(member.min_shear('Fy', 'Combo 1'))
                )
                
                # Calculate stress ratios
                geom = spec.get_geometry()
                mat_props = MATERIAL_STRENGTHS[spec.material]
                
                c = spec.height / 2
                bending_stress = abs(max_moment * c / geom.Iz)
                bending_ratio = bending_stress / mat_props['f_mk']
                
                if spec.shape == 'rectangular':
                    shear_stress = 1.5 * abs(max_shear) / geom.A
                else:  # I-beam
                    catalog_data = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == spec.material_id].iloc[0]
                    web_area = (spec.height - 2*catalog_data['flange_thickness']) * catalog_data['web_thickness']
                    shear_stress = abs(max_shear) / web_area
                
                shear_ratio = shear_stress / mat_props['f_vk']
                
                max_stress_ratio = max(max_stress_ratio, bending_ratio, shear_ratio)
            
            if max_deflection > max_deflection_limit:
                # Penalize configurations that exceed deflection limit
                penalty = 1000 * (max_deflection - max_deflection_limit)
                return penalty + total_cost
            
            if max_stress_ratio > 1.0:
                # Penalize configurations that fail stress checks
                penalty = 5000 * (max_stress_ratio - 1.0)
                return penalty + total_cost
            
            # Normalize metrics for multi-objective optimization
            # Note: These normalization factors should be adjusted based on expected ranges
            normalized_deflection = max_deflection / max_deflection_limit
            
            objective_value = (
                objective_weights['cost'] +
                objective_weights['deflection'] * normalized_deflection
            )
            
            return objective_value
            
        except Exception as e:
            warnings.warn(f"Configuration failed: {str(e)}")
            return 1e6
    
    return objective


def setup_search_space() -> Tuple[list, list]:
    beam_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'beam']['id'].unique().tolist()
    double_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'double']['id'].unique().tolist()
    floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()
    space = [
        Categorical(beam_ids, name='east_material'),
        Integer(1, 3, name='east_quantity'),
        Integer(0, 400, name='east_padding'),
        
        Categorical(beam_ids, name='west_material'),
        Integer(1, 3, name='west_quantity'),
        Integer(0, 400, name='west_padding'),
        
        Categorical(beam_ids, name='tail_material'),
        Integer(1, 3, name='tail_quantity'),
        Integer(0, 400, name='tail_padding'),
        
        Categorical(beam_ids + double_ids, name='trimmer_material'),
        Categorical(beam_ids + double_ids, name='header_material'),  
        Categorical(floor_ids, name='plank_material'),
    ]
    dimension_names = [dim.name for dim in space]
    
    return space, dimension_names


def run_bayesian_optimization(
    n_calls: int = 50,
    n_initial_points: int = 10,
    objective_weights: Dict[str, float] = None,
    max_deflection_limit: float = 10.0,
    random_state: int = 42) -> Dict:

    space, dimension_names = setup_search_space()
    objective_fn = create_optimization_objective(objective_weights, max_deflection_limit)
    
    # Wrapper for use_named_args decorator
    @use_named_args(space)
    def objective_wrapper(**params):
        return objective_fn(params)
    
    result = gp_minimize(
        objective_wrapper,
        space,
        n_calls=n_calls,
        n_initial_points=n_initial_points,
        random_state=random_state,
        verbose=True,
        n_jobs=-1
    )
    
    optimization_summary = {
        'best_params': {name: val for name, val in zip(dimension_names, result.x)},
        'best_objective_value': result.fun,
        'n_calls': len(result.func_vals),
        'all_objective_values': result.func_vals,
        'all_params': result.x_iters,
        'dimension_names': dimension_names,
        'convergence_data': {
            'iterations': list(range(len(result.func_vals))),
            'objective_values': result.func_vals,
            'best_so_far': np.minimum.accumulate(result.func_vals)
        }
    }
    
    return optimization_summary


if __name__ == '__main__':
    # Units are mm, N, and MPa (N/mm²)
    # INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG = prep_data()
    
    results = run_bayesian_optimization(
        design_params=INPUT_PARAMS,
        material_strengths=MATERIAL_STRENGTHS,
        n_calls=100,
        n_initial_points=20,
        objective_weights={'cost': 0.5, 'deflection': 0.3},
        max_deflection_limit=10.0,
    )
    
    best_metrics = evaluate_configuration(
        results['best_params'],
        MATERIAL_CATALOG,
        INPUT_PARAMS,
        MATERIAL_STRENGTHS
    )
    
    print(f"Total cost: ${best_metrics['total_cost']:.2f}")
    print(f"Total volume: {best_metrics['total_volume_m3']:.4f} m³")
    print(f"Max deflection: {best_metrics['max_deflection_mm']:.3f} mm")
    print(f"Max stress ratio: {best_metrics['max_stress_ratio']:.3f}")
    print(f"Passes all checks: {best_metrics['passes_all_checks']}")
    
    results_df = pd.DataFrame([results['best_params']])
    results_df['best_objective'] = results['best_objective_value']
    results_df.to_csv('optimization_results.csv', index=False)

In [ ]:
def prep_data():
    # Units are mm, N, and MPa (N/mm²)
    input_params = pd.read_csv('data/design_parameters.csv').iloc[0].to_dict()
    input_params = DesignParameters(**input_params)

    material_str = pd.read_csv('data/material_strengths.csv').set_index('material').to_dict(orient='index')

    material_catalog = pd.read_csv('data/material_catalog.csv')
    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    # Add double beams
    doubled_beams = material_catalog[(material_catalog['type'] == 'beam') & (material_catalog['material'] == 'c24')].copy()
    doubled_beams['base'] = doubled_beams['base'] * 2
    doubled_beams['type'] = 'double'
    doubled_beams['source'] = doubled_beams['id']
    material_catalog = pd.concat([material_catalog, doubled_beams], ignore_index=True)

    # Standardize floor materials to 200mm width
    is_floor = material_catalog['type'] == 'floor'
    material_catalog.loc[is_floor, 'cost_unit'] = material_catalog.loc[is_floor, 'cost_unit'] * (200 / material_catalog.loc[is_floor, 'base'])
    material_catalog.loc[is_floor, 'base'] = 200

    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    return input_params, material_str, material_catalog

input_params, material_str, material_catalog = prep_data()
material_catalog

# OLD

In [ ]:
span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
k_mod = 0.8
gamma_M = 1.3
service_deflection_ratio = 300
dynamic_factors = [1.0, 1.2, 1.5]

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08), (0.02, 0.04), (0.01, 0.02)]

wood_grades = {
    "C14 (Softwood)": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    "C18 (Softwood)": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    "C22 (Softwood)": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24 (Softwood)": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    "D30 (Hardwood)": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    "D40 (Hardwood)": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
    "D50 (Hardwood)": {"f_mk": 50, "f_vk": 5.0, "E_mean": 16000e6},
}

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    defl_ok = True
    for dyn in dynamic_factors:
        if not defl_ok: break
        w_total = (w_surface * dyn + beam_weight / span) * 1000
        M_max = w_total * span**2 / 8
        V_max = w_total * span / 2
        I = b * h**3 / 12
        c = h / 2
        sigma_max = M_max * c / I
        Q = b * h**2 / 8
        tau_max = V_max * Q / (I * b)
        A = b * h
        tau_avg = V_max / A
        print(f"\nBeam {b*100:.1f}x{h*100:.1f} cm, Dynamic factor={dyn}")
        print(f"Distributed load: {w_total/1000:.3f} kN/m")
        print(f"Max moment: {M_max/1e3:.3f} kN·m")
        print(f"Max shear: {V_max/1e3:.3f} kN")
        print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
        print(f"Shear stress: {tau_max/1e6:.3f} MPa")
        for grade, props in wood_grades.items():
            f_md = k_mod * props["f_mk"] / gamma_M
            f_vd = k_mod * props["f_vk"] / gamma_M
            E = props["E_mean"]
            delta_max = 5 * w_total * span**4 / (384 * E * I)
            service_limit = span / service_deflection_ratio
            bend_ok = sigma_max/1e6 <= f_md
            shear_ok = tau_max/1e6 <= f_vd
            defl_ok = delta_max <= service_limit
            status = []
            status.append("Bend OK" if bend_ok else "BEND FAIL")
            status.append("Shear OK" if shear_ok else "SHEAR FAIL")
            status.append("Defl OK" if defl_ok else "DEFLECTION FAIL")
            print(f"{grade}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, "
                  f"Defl={delta_max:.3f} m, Status: {', '.join(status)}")


In [ ]:
import pandas as pd

# 1. Room & Floor Geometry
room_width_m = 1.87
room_length_m = 3.0

# 2. Loading Conditions
live_loads = range(3,31)
dead_load_planks_q_kPa = 0.25 # Dead load of the floor planks/decking in kN/m^2

# 3. Beam Configuration & Support
number_of_beams = 4
beam_bearing_length_m = 0.04

# 4. Material Properties & Safety Factors
k_mod = 0.8   # Modification factor for load duration and moisture (e.g., 0.8 for service class 2)
gamma_M = 1.3 # Partial safety factor for wood material properties
brick_safety_factor = 5.0 # Higher safety factor for brittle, hollow brick

# 5. Serviceability & Dynamic Limits
service_deflection_ratio = 360 # Stricter deflection limit (L/360 is common)
dynamic_factor = 1.0 # Assuming static load for simplicity. Change if floor will see dynamic use.

beam_sizes_m = [
    # (0.033, 0.07), 
    # (0.045, 0.09), 
    (0.06, 0.12), 
    (0.08, 0.16), 
    # (0.10, 0.20), 
    # (0.12, 0.24), 
]

wood_grades = {
    # "C14": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    # "C18": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    # "C22": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    # "D30": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    # "D40": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
}

wood_species = {
    # "Pinus sylvestris": {"density_kN": 4.707, "durability_DC": 3.5},
    # "Picea abies": {"density_kN": 4.413, "durability_DC": 4},
    "Larix decidua": {"density_kN": 5.884, "durability_DC": 3},
    # "Pinus radiata": {"density_kN": 4.217, "durability_DC": 4.5},
}

brick_types = {
    "Spanish Hollow Brick (Assumed)": {"compressive_strength_MPa": 10.0, "is_hollow": True},
    "Solid Clay Brick (for comparison)": {"compressive_strength_MPa": 20.0, "is_hollow": False},
}
selected_brick = brick_types["Spanish Hollow Brick (Assumed)"]

results = []
span = room_width_m
beam_spacing_m = room_length_m / number_of_beams

for b, h in beam_sizes_m:
    for live_load_q_kPa in live_loads:
        for grade_name, wood_props in wood_grades.items():
            for species_name, species_props in wood_species.items():
                # Self-weight of the beam
                beam_volume = b * h * span
                beam_weight_kN = beam_volume * species_props['density_kN']
                beam_self_weight_dist_kN_m = beam_weight_kN / span
                # Surface loads (live + dead) tributary to one beam
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * beam_spacing_m
                # Total design load on the beam
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000

                # Bending and Shear Forces
                M_max_Nm = w_total_N_m * span**2 / 8
                V_max_N = w_total_N_m * span / 2 # This is also the reaction force at the support
                # Beam geometric properties
                I_m4 = b * h**3 / 12  # Moment of inertia
                A_m2 = b * h           # Cross-sectional area
                # Stresses in the beam
                sigma_max_Pa = (M_max_Nm * (h / 2)) / I_m4
                tau_max_Pa = (3 * V_max_N) / (2 * A_m2) # Max shear for rectangle
                # Allowable design stresses for the wood
                f_md_Pa = (k_mod * wood_props["f_mk"] / gamma_M) * 1e6
                f_vd_Pa = (k_mod * wood_props["f_vk"] / gamma_M) * 1e6
                # Deflection
                E_Pa = wood_props["E_mean"]
                delta_max_m = (5 * w_total_N_m * span**4) / (384 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio

                # Calculate stress on bricks
                bearing_area_m2 = b * beam_bearing_length_m
                bearing_stress_on_brick_Pa = V_max_N / bearing_area_m2
                allowable_brick_stress_Pa = (selected_brick["compressive_strength_MPa"] * 1e6) / brick_safety_factor

                # Failure mode booleans
                bend_ok = sigma_max_Pa <= f_md_Pa
                shear_ok = tau_max_Pa <= f_vd_Pa
                defl_ok = delta_max_m <= service_limit_m
                brick_ok = bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa

                results.append({
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Wood Grade": grade_name,
                    "Wood Species": species_name,
                    "Total Beam Weight": beam_weight_kN * 6,
                    "Beam Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6,
                    "Beam Shear Stress (MPa)": tau_max_Pa / 1e6,
                    "Allowable Shear (MPa)": f_vd_Pa / 1e6,
                    "Live load": live_load_q_kPa,
                    "Deflection (mm)": delta_max_m * 1000,
                    "Allowable Defl (mm)": service_limit_m * 1000,
                    "Brick Bearing Stress (MPa)": bearing_stress_on_brick_Pa / 1e6,
                    "Allowable Brick Stress (MPa)": allowable_brick_stress_Pa / 1e6,
                    "Bend OK": bend_ok,
                    "Shear OK": shear_ok,
                    "Defl OK": defl_ok,
                    "Brick OK": brick_ok
                })

df = pd.DataFrame(results)
df